# Heart Failure Prediction Model

Heart failure clinical recoreds dataset을 이용해 심부전증으로 인한 사망 여부를 예측하는 모델을 만듭니다.


## 학습할 알고리즘

- 디시전 트리
- 랜덤 포레스트
- 로지스틱 회귀


In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

plt.rc("font", family="Malgun Gothic")


def metrics(y_test, pred):
    print(f"정확도: {accuracy_score(y_test,pred):.4f}")
    print(f"정밀도: {precision_score(y_test,pred):.4f}")
    print(f"재현율: {recall_score(y_test,pred):.4f}")
    print(f"F1 스코어: {f1_score(y_test,pred):.4f}")
    print(f"오차행렬: {confusion_matrix(y_test,pred)}")


def append_scores(y_test, pred, scores):
    scores["accuracy"].append(accuracy_score(y_test, pred))
    scores["precision"].append(precision_score(y_test, pred))
    scores["recall"].append(recall_score(y_test, pred))
    scores["f1"].append(f1_score(y_test, pred))


def print_scores(scores):
    print(
        f"""평균 정확도: {np.mean(scores['accuracy']):.4f}
평균 정밀도: {np.mean(scores['precision']):.4f}
평균 재현율: {np.mean(scores['recall']):.4f}
평균 F1 스코어: {np.mean(scores['f1']):.4f}
"""
    )


def k_fold_predict(features, label, model):
    kfold = KFold(n_splits=10, shuffle=True)
    scores = {
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1": [],
    }
    for train_index, test_index in kfold.split(features):
        x_train, x_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = label.iloc[train_index], label.iloc[test_index]
        model.fit(x_train, y_train)
        pred = model.predict(x_test)
        append_scores(y_test, pred, scores)
        confusion_matrix(y_test, pred)
        metrics(y_test, pred)

    print_scores(scores)


def k_fold_scaled_predict(features, label, model):
    kfold = KFold(n_splits=10, shuffle=True)
    scores = {
        "accuracy": [],
        "precision": [],
        "recall": [],
        "f1": [],
    }
    scaler.fit(features)
    for train_index, test_index in kfold.split(features):
        x_train, x_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = label.iloc[train_index], label.iloc[test_index]
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
        model.fit(x_train, y_train)
        pred = model.predict(x_test)
        append_scores(y_test, pred, scores)
        confusion_matrix(y_test, pred)
        metrics(y_test, pred)

    print_scores(scores)


scaler = StandardScaler()

path = "data/heart_failure_clinical_records_dataset.csv"
df = pd.read_csv(path, encoding="utf-8")
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


## 디시전 트리

- 비선형 관계도 학습 가능
- 스케일링 불필요
- 이상치 영향 강함


In [8]:
model = DecisionTreeClassifier()
decision_df = df.copy()
decision_df.drop(
    columns=[
        "diabetes",
        "sex",
        "smoking",
    ],
    inplace=True,
)
decision_df = decision_df[decision_df["serum_sodium"] > 115]
decision_df = decision_df[decision_df["serum_creatinine"] < 4]
features = decision_df.drop("DEATH_EVENT", axis=1)
label = decision_df["DEATH_EVENT"]

In [9]:
k_fold_predict(features, label, model)

정확도: 0.6552
정밀도: 0.5833
재현율: 0.5833
F1 스코어: 0.5833
오차행렬: [[12  5]
 [ 5  7]]
정확도: 0.8276
정밀도: 0.6000
재현율: 0.8571
F1 스코어: 0.7059
오차행렬: [[18  4]
 [ 1  6]]
정확도: 0.8621
정밀도: 0.5000
재현율: 1.0000
F1 스코어: 0.6667
오차행렬: [[21  4]
 [ 0  4]]
정확도: 0.8276
정밀도: 0.8333
재현율: 0.5556
F1 스코어: 0.6667
오차행렬: [[19  1]
 [ 4  5]]
정확도: 0.6552
정밀도: 0.5333
재현율: 0.7273
F1 스코어: 0.6154
오차행렬: [[11  7]
 [ 3  8]]
정확도: 0.7931
정밀도: 0.6250
재현율: 0.6250
F1 스코어: 0.6250
오차행렬: [[18  3]
 [ 3  5]]
정확도: 0.7931
정밀도: 0.5000
재현율: 1.0000
F1 스코어: 0.6667
오차행렬: [[17  6]
 [ 0  6]]
정확도: 0.8621
정밀도: 0.8000
재현율: 0.8000
F1 스코어: 0.8000
오차행렬: [[17  2]
 [ 2  8]]
정확도: 0.8621
정밀도: 0.8182
재현율: 0.8182
F1 스코어: 0.8182
오차행렬: [[16  2]
 [ 2  9]]
정확도: 0.7241
정밀도: 0.7000
재현율: 0.5833
F1 스코어: 0.6364
오차행렬: [[14  3]
 [ 5  7]]
평균 정확도: 0.7862
평균 정밀도: 0.6493
평균 재현율: 0.7550
평균 F1 스코어: 0.6784



## 랜덤 포레스트

- 여러개의 결정 트리를 조합하여 예측하는 앙상블 학습 모델입니다.
- 각각의 결정 트리는 훈련 데이터의 일부만 랜덤하게 학습합니다.
- 최종 예측은 분류의 경우 다수결 투표, 회귀의 경우 평균을 사용합니다.


In [10]:
model = RandomForestClassifier()
random_df = df.copy()
random_df.drop(
    columns=[
        "diabetes",
        "sex",
        "smoking",
    ],
    inplace=True,
)
features = random_df.drop("DEATH_EVENT", axis=1)
label = random_df["DEATH_EVENT"]

In [11]:
k_fold_predict(features, label, model)

정확도: 1.0000
정밀도: 1.0000
재현율: 1.0000
F1 스코어: 1.0000
오차행렬: [[21  0]
 [ 0  9]]
정확도: 0.7333
정밀도: 1.0000
재현율: 0.5000
F1 스코어: 0.6667
오차행렬: [[14  0]
 [ 8  8]]
정확도: 0.7667
정밀도: 0.7143
재현율: 0.5000
F1 스코어: 0.5882
오차행렬: [[18  2]
 [ 5  5]]
정확도: 0.8333
정밀도: 0.5556
재현율: 0.8333
F1 스코어: 0.6667
오차행렬: [[20  4]
 [ 1  5]]
정확도: 0.8667
정밀도: 0.7273
재현율: 0.8889
F1 스코어: 0.8000
오차행렬: [[18  3]
 [ 1  8]]
정확도: 0.7333
정밀도: 0.5714
재현율: 0.8000
F1 스코어: 0.6667
오차행렬: [[14  6]
 [ 2  8]]
정확도: 0.8000
정밀도: 0.6667
재현율: 0.2857
F1 스코어: 0.4000
오차행렬: [[22  1]
 [ 5  2]]
정확도: 0.8000
정밀도: 0.6364
재현율: 0.7778
F1 스코어: 0.7000
오차행렬: [[17  4]
 [ 2  7]]
정확도: 0.8667
정밀도: 1.0000
재현율: 0.6667
F1 스코어: 0.8000
오차행렬: [[18  0]
 [ 4  8]]
정확도: 0.8621
정밀도: 0.7000
재현율: 0.8750
F1 스코어: 0.7778
오차행렬: [[18  3]
 [ 1  7]]
평균 정확도: 0.8262
평균 정밀도: 0.7572
평균 재현율: 0.7127
평균 F1 스코어: 0.7066



## 로지스틱 회귀

- 선형 모델로서, 독립 변수와 종속 변수 사이의 선형 관계를 가정
- 선형 관계가 강한 피처를 우선 선택
- 스케일링 필수


In [12]:
logistic_df = df.copy()
logistic_df.drop(
    columns=[
        "diabetes",
        "sex",
        "smoking",
        "platelets",
        "ejection_fraction",
        "creatinine_phosphokinase",
    ],
    inplace=True,
)
model = LogisticRegression()
features = logistic_df.drop("DEATH_EVENT", axis=1)
label = logistic_df["DEATH_EVENT"]
k_fold_scaled_predict(features, label, model)

정확도: 0.8000
정밀도: 0.8333
재현율: 0.5000
F1 스코어: 0.6250
오차행렬: [[19  1]
 [ 5  5]]
정확도: 0.8000
정밀도: 0.6667
재현율: 0.5000
F1 스코어: 0.5714
오차행렬: [[20  2]
 [ 4  4]]
정확도: 0.8000
정밀도: 1.0000
재현율: 0.5000
F1 스코어: 0.6667
오차행렬: [[18  0]
 [ 6  6]]
정확도: 0.7333
정밀도: 0.5556
재현율: 0.5556
F1 스코어: 0.5556
오차행렬: [[17  4]
 [ 4  5]]
정확도: 0.7333
정밀도: 0.8750
재현율: 0.5000
F1 스코어: 0.6364
오차행렬: [[15  1]
 [ 7  7]]
정확도: 0.8333
정밀도: 0.6364
재현율: 0.8750
F1 스코어: 0.7368
오차행렬: [[18  4]
 [ 1  7]]
정확도: 0.8333
정밀도: 0.7273
재현율: 0.8000
F1 스코어: 0.7619
오차행렬: [[17  3]
 [ 2  8]]
정확도: 0.8000
정밀도: 0.6364
재현율: 0.7778
F1 스코어: 0.7000
오차행렬: [[17  4]
 [ 2  7]]
정확도: 0.9333
정밀도: 1.0000
재현율: 0.7500
F1 스코어: 0.8571
오차행렬: [[22  0]
 [ 2  6]]
정확도: 0.8966
정밀도: 0.8571
재현율: 0.7500
F1 스코어: 0.8000
오차행렬: [[20  1]
 [ 2  6]]
평균 정확도: 0.8163
평균 정밀도: 0.7788
평균 재현율: 0.6508
평균 F1 스코어: 0.6911

